# Package Imports

In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

% matplotlib inline

import plotly
plotly.offline.init_notebook_mode(connected=True)

import sys
sys.path.append("..")

In [ ]:
import thor

config = thor.Config()

In [ ]:
DATABASE = "../data/msst_survey.db"
con = sql.connect(DATABASE)

## Config

In [ ]:
from thor import propagateTestParticle

## Plotting Code

In [ ]:
from thor.plotting import plotProjections
from thor.plotting import plotProjections3D
from thor.plotting import plotObservations
from thor.plotting import plotObservations3D
from thor.plotting import plotBinnedContour
from thor.plotting import plotScatterContour
from thor.plotting import plotCell
from thor.plotting import _setPercentage

## Classes 

In [ ]:
from thor import Cell
from thor import TestParticle

## Functions

In [ ]:
from thor import findAverageObject
from thor import findExposureTimes
from thor import buildCellForVisit
from thor import rangeAndShift
from thor import clusterAndLink
from thor import analyzeClusters
from thor import analyzeProjections
from thor import runRangeAndShiftOnVisit
from thor import runClusterAndLinkOnVisit

# Load Data

In [ ]:
observationsNoNoise = pd.read_sql("""SELECT * FROM observations""", con)
print(len(observationsNoNoise))
observationsNoNoise.drop_duplicates(subset=["designation", "exp_mjd"], inplace=True)
print(len(observationsNoNoise))
noise = pd.read_sql("""SELECT * FROM noise_100""", con)
noise["obsId"] = np.arange(observationsNoNoise["obsId"].values[-1] + 1, observationsNoNoise["obsId"].values[-1] + 1 + len(noise))

In [ ]:
observations = pd.concat([observationsNoNoise, noise], sort=False)
observations.reset_index(inplace=True, drop=True)
del observationsNoNoise
del noise

In [ ]:
survey = pd.read_sql("""SELECT * FROM survey""", con)

In [ ]:
orbits = pd.read_sql("""SELECT * FROM mpcOrbitCat""", con)
# Only grab the orbits of objects with observations
orbits = orbits[orbits["designation"].isin(observations["designation"].unique())]

In [ ]:
neos = orbits[orbits["a_au"] <= 1.3]["designation"].values

## Range and Shift

In [ ]:
projected_obs, average_obj = runRangeAndShiftOnVisit(observations, 
                        1, 
                        None,
                        None,
                        useAverageObject=True,
                        searchArea=10,
                        cellArea=10,
                        verbose=False)

### Analyze Projections

In [ ]:
allObjects, summary = analyzeProjections(projected_obs)

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=200)
ax.errorbar(allObjects[allObjects["findable"] == 1]["dtheta_x/dt_median"].values, 
            allObjects[allObjects["findable"] == 1]["dtheta_y/dt_median"].values,
            yerr=allObjects[allObjects["findable"] == 1]["dtheta_y/dt_sigma"].values,
            xerr=allObjects[allObjects["findable"] == 1]["dtheta_x/dt_sigma"].values,
            fmt="o",
            ms=0.01,
            capsize=0.1,
            elinewidth=0.1,
            c="k")
ax.set_xlabel(r"Median $ d\theta_X / dt$ [Degrees Per Day]")
ax.set_ylabel(r"Median $ d\theta_Y / dt$ [Degrees Per Day]")
rect = patches.Rectangle((-0.1,-0.1),0.2,0.2,linewidth=0.5,edgecolor='r',facecolor='none')
ax.add_patch(rect)
ax.set_title("Findable Objects")
ax.text(_setPercentage(ax.get_xlim(), 0.05), _setPercentage(ax.get_ylim(), 0.1), "Objects: {}".format(len(allObjects[allObjects["findable"] == 1])))
ax.text(_setPercentage(ax.get_xlim(), 0.05), _setPercentage(ax.get_ylim(), 0.04), "Objects in Grid: {}".format(len(allObjects[in_zone_findable])), color="r")
#fig.savefig("../analysis/msst/plots/findable_projection_200bins_0005eps.png")

## Cluster and Link

In [ ]:
allClusters, clusterMembers = clusterAndLink(
        projected_obs,
        eps=0.005, 
        minSamples=5, 
        vxRange=[-0.1, 0.1], 
        vyRange=[-0.1, 0.1],
        vxBins=200,
        vyBins=200, 
        threads=5)

## Analysis

In [ ]:
allClusters, clusterMembers, allObjects, summary = analyzeClusters(
    projected_obs,
    allClusters, 
    clusterMembers, 
    allObjects,
    summary,
    minSamples=5, 
    partialThreshold=0.8)

In [ ]:
in_zone_findable = ((allObjects["dtheta_x/dt_median"] >= -0.1) 
 & (allObjects["dtheta_x/dt_median"] <= 0.1) 
 & (allObjects["dtheta_y/dt_median"] <= 0.1) 
 & (allObjects["dtheta_y/dt_median"] >= -0.1)
 & (allObjects["findable"] == 1))

in_zone_missed = ((allObjects["dtheta_x/dt_median"] >= -0.1) 
 & (allObjects["dtheta_x/dt_median"] <= 0.1) 
 & (allObjects["dtheta_y/dt_median"] <= 0.1) 
 & (allObjects["dtheta_y/dt_median"] >= -0.1)
 & (allObjects["findable"] == 1)
 & (allObjects["found"] == 0))

in_smallzone_missed = ((allObjects["dtheta_x/dt_median"] >= -0.03) 
 & (allObjects["dtheta_x/dt_median"] <= 0.03) 
 & (allObjects["dtheta_y/dt_median"] <= 0.03) 
 & (allObjects["dtheta_y/dt_median"] >= -0.03)
 & (allObjects["findable"] == 1)
 & (allObjects["found"] == 0))

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=200)
ax.errorbar(allObjects[(allObjects["found"] == 1)]["dtheta_x/dt_median"].values, 
            allObjects[(allObjects["found"] == 1)]["dtheta_y/dt_median"].values,
            yerr=allObjects[(allObjects["found"] == 1)]["dtheta_y/dt_sigma"].values,
            xerr=allObjects[(allObjects["found"] == 1) ]["dtheta_x/dt_sigma"].values,
            fmt="o",
            ms=0.01,
            capsize=0.1,
            elinewidth=0.1,
            c="b")

rect = patches.Rectangle((-0.1,-0.1),0.2,0.2,linewidth=0.5,edgecolor='r',facecolor='none')
ax.add_patch(rect)
ax.set_xlabel(r"Median $ d\theta_X / dt$ [Degrees Per Day]")
ax.set_ylabel(r"Median $ d\theta_Y / dt$ [Degrees Per Day]")

ax.set_xlabel(r"Median $ d\theta_X / dt$ [Degrees Per Day]")
ax.set_ylabel(r"Median $ d\theta_Y / dt$ [Degrees Per Day]")
ax.set_title("Found Objects")
ax.set_xlim(-0.11, 0.11)
ax.set_ylim(-0.11, 0.11)
ax.set_aspect("equal")
ax.text(_setPercentage(ax.get_xlim(), 0.06), _setPercentage(ax.get_ylim(), 0.1), "Objects: {}".format(len(allObjects[allObjects["found"] == 1])))
#fig.savefig("../analysis/msst/plots/found_projection_200bins_0007eps.png")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=200)
ax.errorbar(allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_x/dt_median"].values, 
            allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_y/dt_median"].values,
            yerr=allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_y/dt_sigma"].values,
            xerr=allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_x/dt_sigma"].values,
            fmt="o",
            ms=0.01,
            capsize=0.1,
            elinewidth=0.1,
            c="k")

rect = patches.Rectangle((-0.1,-0.1),0.2,0.2,linewidth=0.5,edgecolor='r',facecolor='none')
ax.add_patch(rect)
ax.set_xlabel(r"Median $ d\theta_X / dt$ [Degrees Per Day]")
ax.set_ylabel(r"Median $ d\theta_Y / dt$ [Degrees Per Day]")

ax.set_xlabel(r"Median $ d\theta_X / dt$ [Degrees Per Day]")
ax.set_ylabel(r"Median $ d\theta_Y / dt$ [Degrees Per Day]")
ax.set_title("Missed Objects")
ax.set_aspect("equal")
ax.text(_setPercentage(ax.get_xlim(), 0.05), _setPercentage(ax.get_ylim(), 0.1), "Objects: {}".format(len(allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)])))
ax.text(_setPercentage(ax.get_xlim(), 0.05), _setPercentage(ax.get_ylim(), 0.04), "Objects in Grid: {}".format(len(allObjects[in_zone_missed])), color="r")
#fig.savefig("../analysis/msst/plots/missed_projection_200bins_0007eps.png")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=200)
ax.errorbar(allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_x/dt_median"].values, 
            allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_y/dt_median"].values,
            yerr=allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_y/dt_sigma"].values,
            xerr=allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_x/dt_sigma"].values,
            fmt="o",
            ms=0.01,
            capsize=0.1,
            elinewidth=0.1,
            c="k")

rect = patches.Rectangle((-0.1,-0.1),0.2,0.2,linewidth=0.5,edgecolor='r',facecolor='none')
ax.add_patch(rect)
ax.set_xlabel(r"Median $ d\theta_X / dt$ [Degrees Per Day]")
ax.set_ylabel(r"Median $ d\theta_Y / dt$ [Degrees Per Day]")
ax.set_title("Missed Objects")

ax.set_xlim(-0.11, 0.11)
ax.set_ylim(-0.11, 0.11)
ax.set_aspect("equal")
#fig.savefig("../analysis/msst/plots/missed_zoom_projection_200bins_0007eps.png")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=300)
xbins = np.linspace(-0.1, 0.1, num=400)
ybins = np.linspace(-0.1, 0.1, num=400)
for xb, yb in zip(xbins, ybins):
    ax.hlines(yb, -0.1, 0.1, lw=0.5, alpha=0.5)
    ax.vlines(xb, -0.1, 0.1, lw=0.5, alpha=0.5)
ax.errorbar(allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_x/dt_median"].values, 
            allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_y/dt_median"].values,
            yerr=allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_y/dt_sigma"].values,
            xerr=allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_x/dt_sigma"].values,
            fmt="o",
            ms=0.01,
            capsize=0.1,
            elinewidth=0.1,
            c="r")

rect = patches.Rectangle((-0.1,-0.1),0.2,0.2,linewidth=0.5,edgecolor='r',facecolor='none')
ax.add_patch(rect)
ax.set_xlabel(r"Median $ d\theta_X / dt$ [Degrees Per Day]")
ax.set_ylabel(r"Median $ d\theta_Y / dt$ [Degrees Per Day]")
ax.set_title("Missed Objects")

ax.set_xlim(-0.11, 0.11)
ax.set_ylim(-0.11, 0.11)
ax.set_aspect("equal")
#fig.savefig("../analysis/msst/plots/missed_grid_projection_200bins_0007eps.png")


In [ ]:
fig, ax = plt.subplots(1, 1, dpi=300)
xbins = np.linspace(-0.1, 0.1, num=400)
ybins = np.linspace(-0.1, 0.1, num=400)
xbin_width = xbins[1] - xbins[0]
ybin_width = ybins[1] - ybins[0]

for xb, yb in zip(xbins, ybins):
    ax.hlines(yb, -0.1, 0.1, lw=0.5, alpha=0.5)
    ax.vlines(xb, -0.1, 0.1, lw=0.5, alpha=0.5)
ax.errorbar(allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_x/dt_median"].values, 
            allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_y/dt_median"].values,
            yerr=allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_y/dt_sigma"].values,
            xerr=allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)]["dtheta_x/dt_sigma"].values,
            fmt="o",
            ms=0.5,
            capsize=0.1,
            elinewidth=0.1,
            c="r")

rect = patches.Rectangle((-0.1,-0.1),0.2,0.2,linewidth=0.5,edgecolor='r',facecolor='none')
ax.add_patch(rect)
ax.set_xlabel(r"Median $ d\theta_X / dt$ [Degrees Per Day]")
ax.set_ylabel(r"Median $ d\theta_Y / dt$ [Degrees Per Day]")
ax.set_title("Missed Objects")

ax.set_xlim(-0.03, 0.03)
ax.set_ylim(-0.03, 0.03)
ax.set_aspect("equal")
#fig.savefig("../analysis/msst/plots/missed_zoom_grid_200bins_0007eps.png")

In [ ]:
allObjects[in_smallzone_missed]["designation"].values

In [ ]:
fig = plotProjections(projected_obs[projected_obs["designation"].isin(allObjects[in_smallzone_missed]["designation"].values)])

In [ ]:
fig = plotProjections3D(projected_obs[projected_obs["designation"].isin(allObjects[in_smallzone_missed]["designation"].values)])

In [ ]:
allObjects[in_smallzone_missed]["dtheta_x/dt_median"].values

In [ ]:
allClusters_debug, clusterMembers_debug = clusterAndLink(
        projected_obs,
        eps=0.001, 
        minSamples=5, 
        vxValues=allObjects[in_smallzone_missed]["dtheta_x/dt_median"].values,
        vyValues=allObjects[in_smallzone_missed]["dtheta_y/dt_median"].values,
        #vxRange=[-0.1, 0.1], 
        #vyRange=[-0.1, 0.1],
        #vxBins=200,
        #vyBins=200, 
        threads=5)

In [ ]:
allClusters_debug, clusterMembers_debug, allObjects_debug, summary_debug = analyzeClusters(
    projected_obs,
    allClusters_debug, 
    clusterMembers_debug, 
    allObjects_debug,
    summary_debug,
    minSamples=5, 
    partialThreshold=0.8)

In [ ]:
#aprojected_obs.to_csv("project_obs_200x200_visit1.csv", index=False, sep=" ")

In [ ]:
visitId = 1
columnMapping = config.columnMapping
avg_obj = average_obj[columnMapping["name"]].values[0]
o = orbits[orbits[columnMapping["name"]] == avg_obj]
    
found = orbits[orbits[columnMapping["name"]].isin(allObjects[allObjects["found"] == 1][columnMapping["name"]])]
missed = orbits[orbits[columnMapping["name"]].isin((allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)][columnMapping["name"]]))]

In [ ]:
fig, ax = plotScatterContour(missed, 
                             columnMapping["a_au"],
                             columnMapping["i_deg"],
                             columnMapping["e"],
                             plotCounts=False, 
                             logCounts=True, 
                             countLevels=4, 
                             mask=None,
                             xLabel="a [AU]",
                             yLabel="i [deg]",
                             zLabel="e",
                             scatterKwargs={"s": 1, "vmin": 0, "vmax": 1})
ax.scatter(o["a_au"].values, o["i_deg"].values, c="r", s=20, marker="+")
ax.set_title("Missed Orbits\nVisit: {}, Object: {}".format(visitId, avg_obj))
ax.text(_setPercentage(ax.get_xlim(), 0.6), _setPercentage(ax.get_ylim(), 0.95), "Missed objects: {}".format(len(missed)))


In [ ]:
fig, ax = plotScatterContour(found, 
                             columnMapping["a_au"],
                             columnMapping["i_deg"],
                             columnMapping["e"],
                             plotCounts=False, 
                             logCounts=True, 
                             countLevels=4, 
                             mask=None,
                             xLabel="a [AU]",
                             yLabel="i [deg]",
                             zLabel="e",
                             scatterKwargs={"s": 1, "vmin": 0, "vmax": 1})        
ax.scatter(o["a_au"].values, o["i_deg"].values, c="r", s=20, marker="+")
ax.set_title("Recovered Orbits\nVisit: {}, Object: {}".format(visitId, avg_obj))
ax.text(_setPercentage(ax.get_xlim(), 0.6), _setPercentage(ax.get_ylim(), 0.95), "Missed objects: {}".format(len(found)))


In [ ]:
found_obs = projected_obs[projected_obs[columnMapping["name"]].isin(allObjects[allObjects["found"] == 1][columnMapping["name"]])]
missed_obs = projected_obs[projected_obs[columnMapping["name"]].isin((allObjects[(allObjects["found"] == 0) & (allObjects["findable"] == 1)][columnMapping["name"]]))]
obs = projected_obs[projected_obs[columnMapping["name"]] == avg_obj]

In [ ]:
fig, ax = plotScatterContour(projected_obs, 
                                 columnMapping["obj_dx/dt_au_p_day"], 
                                 columnMapping["obj_dy/dt_au_p_day"], 
                                 columnMapping["obj_dz/dt_au_p_day"],
                                 countLevels=4, 
                                 xLabel="dx/dt [AU per day]",
                                 yLabel="dy/dt [AU per day]",
                                 zLabel="dz/dt [AU per day]")
   
#ax.text(_setPercentage(ax.get_xlim(), 0.6), _setPercentage(ax.get_ylim(), 0.95), "Missed objects: {}".format(len(missed)))
ax.scatter(*obs[[columnMapping["obj_dx/dt_au_p_day"], columnMapping["obj_dy/dt_au_p_day"]]].values.T, c="r", s=1, marker="+")
ax.set_title("All Orbits\nVisit: {}, Object: {}".format(visitId, avg_obj))

In [ ]:
fig, ax = plotScatterContour(missed_obs, 
                                 columnMapping["obj_dx/dt_au_p_day"], 
                                 columnMapping["obj_dy/dt_au_p_day"], 
                                 columnMapping["obj_dz/dt_au_p_day"],
                                 countLevels=4, 
                                 xLabel="dx/dt [AU per day]",
                                 yLabel="dy/dt [AU per day]",
                                 zLabel="dz/dt [AU per day]")
   
ax.text(_setPercentage(ax.get_xlim(), 0.6), _setPercentage(ax.get_ylim(), 0.95), "Missed objects: {}".format(len(missed)))
ax.scatter(*obs[[columnMapping["obj_dx/dt_au_p_day"], columnMapping["obj_dy/dt_au_p_day"]]].values.T, c="r", s=1, marker="+")
ax.set_title("Missed Orbits\nVisit: {}, Object: {}".format(visitId, avg_obj))

In [ ]:
fig, ax = plotScatterContour(found_obs, 
                                 columnMapping["obj_dx/dt_au_p_day"], 
                                 columnMapping["obj_dy/dt_au_p_day"], 
                                 columnMapping["obj_dz/dt_au_p_day"],
                                 countLevels=4, 
                                 xLabel="dx/dt [AU per day]",
                                 yLabel="dy/dt [AU per day]",
                                 zLabel="dz/dt [AU per day]")
   
ax.scatter(*obs[[columnMapping["obj_dx/dt_au_p_day"], columnMapping["obj_dy/dt_au_p_day"]]].values.T, c="r", s=1, marker="+")
ax.set_title("Found Orbits\nVisit: {}, Object: {}".format(visitId, avg_obj))
ax.text(_setPercentage(ax.get_xlim(), 0.6), _setPercentage(ax.get_ylim(), 0.95), "Found objects: {}".format(len(found)))

In [ ]:
colorByObject = False
dataframe = missed
data = []
if colorByObject is True:
    for name in dataframe[columnMapping["name"]].unique():
        obj = dataframe[dataframe[columnMapping["name"]] == name]
        if name == "NS":
            trace = plotly.graph_objs.Scatter(
                x=obj[columnMapping["RA_deg"]],
                y=obj[columnMapping["Dec_deg"]],
                name=name,
                mode="markers",
                marker=dict(size=2))
        else:
            trace = plotly.graph_objs.Scatter(
                x=obj[columnMapping["RA_deg"]],
                y=obj[columnMapping["Dec_deg"]],
                name=name,
                mode="markers",
                marker=dict(size=2))
        data.append(trace)
else:
    trace = plotly.graph_objs.Scatter(
        x=dataframe[columnMapping["a_au"]],
        y=dataframe[columnMapping["i_deg"]],
        mode="markers",
        text=dataframe[columnMapping["name"]],
        marker=dict(size=2)
    )
    data.append(trace)
    
data.append(plotly.graph_objs.Scatter(
        x=dataframe[dataframe["designation"].isin(["p3143"])][columnMapping["a_au"]],
        y=dataframe[dataframe["designation"].isin(["p3143"])][columnMapping["i_deg"]],
        mode="markers",
        text=dataframe[dataframe["designation"].isin(["p3143"])][columnMapping["name"]],
        marker=dict(size=5)
    ))

layout = dict(
    width=550,
    height=550,
    autosize=False,
    title="",
    scene=dict(
        xaxis=dict(
            title="RA [deg]",
        ),
        yaxis=dict(
            title="Dec [deg]",
        ),
        aspectratio = dict(x=1, y=1)))

fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [ ]:
ooi = ["h7943", "d1608", "K13A77A", "U0752", "K07RM5N", "K9320", "p3143", "25204", "i933", "K06QA2V", "p0707", "K18B09U"]
projected_obs_ooi = projected_obs[projected_obs["designation"].isin(ooi)]
fig = plotObservations(projected_obs_ooi)

In [ ]:
projected_obs_ooi[projected_obs_ooi["designation"] == "d1608"]

In [ ]:
observations[observations["designation"] == "d1608"]

In [ ]:
observations[observations["designation"] == "p3143"]

In [ ]:
fig = plotObservations(observations[observations["designation"].isin(["p3143", "d1608"])])

In [ ]:
allClusters_ooi, clusterMembers_ooi = clusterAndLink(
        projected_obs_ooi,
        eps=0.005, 
        minSamples=5, 
        vxRange=[-1, 1], 
        vyRange=[-1, 1],
        vxBins=1000,
        vyBins=1000, 
        threads=40)

In [ ]:
allClusters_ooi, clusterMembers_ooi, allObjects_ooi, summary_ooi = analyzeClusters(
    projected_obs_ooi,
    allClusters_ooi, 
    clusterMembers_ooi, 
    minSamples=5, 
    partialThreshold=1.0)

## Additional Plots

In [ ]:
fig = plotProjections(
    projected_obs[projected_obs["designation"].isin(allObjects[allObjects["found"] == 1]["designation"].values)],
    colorByObject=False)

In [ ]:
fig = plotProjections(
    projected_obs[projected_obs["designation"].isin(allObjects[(allObjects["findable"] == 1) & (allObjects["found"] == 0)]["designation"].values)],
    colorByObject=False)

In [ ]:
fig = plotProjections(
    projected_obs[projected_obs["designation"].isin(allObjects[(allObjects["findable"] == 1) & (allObjects["found"] == 0)]["designation"].values)],
    colorByObject=True)

In [ ]:
fig = plotProjections(
    projected_obs[projected_obs["obsId"].isin(clusterMembers[clusterMembers["cluster_id"].isin(allClusters[allClusters["pure"] == 1]["cluster_id"].values)]["obs_id"])],
    colorByObject=False)

In [ ]:
fig = plotProjections3D(
    projected_obs[projected_obs["obsId"].isin(clusterMembers[clusterMembers["cluster_id"].isin(allClusters[allClusters["pure"] == 1]["cluster_id"].values)]["obs_id"])],
    colorByObject=False)

In [ ]:
fig = plotProjections3D(
    projected_obs[projected_obs["obsId"].isin(clusterMembers[clusterMembers["cluster_id"].isin([allClusters[allClusters["num_visits"] != allClusters["num_obs"]]["cluster_id"].values[0]])]["obs_id"])],
    colorByObject=False)

In [ ]:
fig = plotProjections3D(
    projected_obs[projected_obs["obsId"].isin(clusterMembers[clusterMembers["cluster_id"].isin(allClusters[allClusters["pure"] == 1]["cluster_id"].values)]["obs_id"])],
    colorByObject=False)